In [1]:
import openmm

In [23]:
platform = openmm.Platform.getPlatformByName('CUDA')

In [24]:
force = openmm.NonbondedForce()
force.setCutoffDistance(2.0)
force.setNonbondedMethod(force.Ewald)
force.addParticle(0.0, 1.0, 0.5)
force.addParticle(1.0, 0.5, 0.6)
force.addParticle(-1.0, 2.0, 0.7)
force.addParticle(0.5, 2.0, 0.8)
force.addParticle(-0.5, 2.0, 0.8)
force.setReciprocalSpaceForceGroup(2)

In [25]:
sliced = openmm.NonbondedForce()
sliced.setCutoffDistance(2.0)
sliced.setNonbondedMethod(sliced.Ewald)
sliced.addParticle(0.0, 1.0, 0.5)
sliced.addParticle(1.0, 0.5, 0.6)
sliced.addParticle(-1.0, 2.0, 0.7)
sliced.addParticle(0.5, 2.0, 0.8)
sliced.addParticle(-0.5, 2.0, 0.8)
sliced.setForceGroup(1)
sliced.setReciprocalSpaceForceGroup(3)

In [26]:
N = force.getNumParticles()

system = openmm.System()
L = float(N)
system.setDefaultPeriodicBoxVectors(openmm.Vec3(L, 0, 0), openmm.Vec3(0, L, 0), openmm.Vec3(0, 0, L))
for i in range(N):
    system.addParticle(1.0)

system.addForce(force)
system.addForce(sliced)

integrator = openmm.VerletIntegrator(0.001)
context = openmm.Context(system, integrator, platform)

In [27]:
positions = []
for i in range(N):
    positions.append(openmm.Vec3(i, 0, 0))

context.setPositions(positions)

state1 = context.getState(getForces=True, getEnergy=True, groups=1<<0)
state2 = context.getState(getForces=True, getEnergy=True, groups=1<<1)

In [28]:
print(state1.getPotentialEnergy(), state2.getPotentialEnergy())

25279.94735808304 kJ/mol 25279.94735808304 kJ/mol


In [29]:
state3 = context.getState(getForces=True, getEnergy=True, groups=1<<2)
state4 = context.getState(getForces=True, getEnergy=True, groups=1<<3)

In [30]:
print(state3.getPotentialEnergy(), state4.getPotentialEnergy())

-191.55956131826053 kJ/mol -191.55956131826053 kJ/mol


In [31]:
del context

In [2]:
import numpy as np

In [3]:
cutoff = 35
lambda_lj = 1
value = 0
lambda_coulomb = [1, value, value*value]

In [4]:
def charge(i):
    return 1-2*(i%2)

In [5]:
positions = np.array([
[12.4167, 12.4167, 12.4167],
[11.9167, 11.9167, 11.9167],
[36.25, 12.4167, 12.4167],
[36.75, 11.9167, 11.9167],
[61.0833, 12.4167, 12.4167],
[60.5833, 11.9167, 11.9167],
[12.4167, 36.25, 12.4167],
[11.9167, 36.75, 11.9167],
[36.25, 36.25, 12.4167],
[36.75, 36.75, 11.9167],
[61.0833, 36.25, 12.4167],
[60.5833, 36.75, 11.9167],
[12.4167, 61.0833, 12.4167],
[11.9167, 60.5833, 11.9167],
[36.25, 61.0833, 12.4167],
[36.75, 60.5833, 11.9167],
[61.0833, 61.0833, 12.4167],
[60.5833, 60.5833, 11.9167],
[12.4167, 12.4167, 36.25],
[11.9167, 11.9167, 36.75],
])
positions

array([[12.4167, 12.4167, 12.4167],
       [11.9167, 11.9167, 11.9167],
       [36.25  , 12.4167, 12.4167],
       [36.75  , 11.9167, 11.9167],
       [61.0833, 12.4167, 12.4167],
       [60.5833, 11.9167, 11.9167],
       [12.4167, 36.25  , 12.4167],
       [11.9167, 36.75  , 11.9167],
       [36.25  , 36.25  , 12.4167],
       [36.75  , 36.75  , 11.9167],
       [61.0833, 36.25  , 12.4167],
       [60.5833, 36.75  , 11.9167],
       [12.4167, 61.0833, 12.4167],
       [11.9167, 60.5833, 11.9167],
       [36.25  , 61.0833, 12.4167],
       [36.75  , 60.5833, 11.9167],
       [61.0833, 61.0833, 12.4167],
       [60.5833, 60.5833, 11.9167],
       [12.4167, 12.4167, 36.25  ],
       [11.9167, 11.9167, 36.75  ]])

In [6]:
distances = np.linalg.norm(positions[:, None, :] - positions[None, :, :], axis=-1)
distances

array([[ 0.        ,  0.8660254 , 23.8333    , 24.34357182, 48.6666    ,
        48.17179004, 23.8333    , 24.34357182, 33.7053761 , 34.41611509,
        54.18915154, 53.9664789 , 48.6666    , 48.17179004, 54.18915154,
        53.9664789 , 68.82496575, 68.119694  , 23.8333    , 24.34357182],
       [ 0.8660254 ,  0.        , 24.34357182, 24.8333    , 49.17168449,
        48.6666    , 24.34357182, 24.8333    , 34.41611509, 35.11958966,
        54.86086077, 54.63635003, 49.17168449, 48.6666    , 54.86086077,
        54.63635003, 69.53387024, 68.82496575, 24.34357182, 24.8333    ],
       [23.8333    , 24.34357182,  0.        ,  0.8660254 , 24.8333    ,
        24.34357182, 33.7053761 , 34.41611509, 23.8333    , 24.34357182,
        34.41974692, 34.41611509, 54.18915154, 53.9664789 , 48.6666    ,
        48.17179004, 54.63635003, 53.9664789 , 33.7053761 , 34.41611509],
       [24.34357182, 24.8333    ,  0.8660254 ,  0.        , 24.34357182,
        23.8333    , 34.41611509, 35.11958966, 2

In [7]:
subsets = [0] * distances.shape[0]
for i in [0, 1, 2, 3, 7, 9, 10, 11, 16, 17, 19]:
    subsets[i] = 1
subsets

[1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1]

In [8]:
# Lennard-Jones
energy_lj = 0
eps = 1e-50
for i in range(distances.shape[0]):
    si = subsets[i]
    for j in range(i+1, distances.shape[0]):
        sj = subsets[j]
        if distances[i, j] < cutoff:
            factor = lambda_lj if (si != sj or si == 1) else 1
            energy_lj += factor*4*eps*((1/distances[i, j])**12 - (1/distances[i, j])**6)
energy_lj

1.2993141180545943e-48

In [9]:
ONE_4PI_EPS0 = 138.935457644382
REACTION_FIELD_K = 0.0
REACTION_FIELD_C = 1/cutoff
energy_coulomb = 0
forces = np.zeros((distances.shape[0], 3))
computed_energies = {}
for i in range(distances.shape[0]):
    si = subsets[i]
    for j in range(i+1, distances.shape[0]):
        sj = subsets[j]
        slice = si*(si+1)//2+sj if si > sj else sj*(sj+1)//2+si
        r2 = distances[i, j]**2
        if distances[i, j] < cutoff:
            prefactor = lambda_coulomb[slice]*ONE_4PI_EPS0*charge(i)*charge(j)
            eij = prefactor*(1.0/distances[i, j] + REACTION_FIELD_K*r2 - REACTION_FIELD_C)
            energy_coulomb += eij
            fij = prefactor*(1.0/distances[i, j]**3 + REACTION_FIELD_K)
            nij = positions[i] - positions[j]
            # if i == 4 or j == 4:
            #     print(f"fij[{i},{j}]: {fij} (dist: {distances[i, j]})")
            computed_energies[(i,j)] = eij
            print(f"({i},{j}): {eij}")
            forces[i] += fij*nij
            forces[j] -= fij*nij
energy_coulomb

(0,1): -0.0
(0,2): 0.0
(0,3): -0.0
(0,6): 0.0
(0,7): -0.0
(0,8): 0.0
(0,9): -0.0
(0,18): 0.0
(0,19): -0.0
(1,2): -0.0
(1,3): 0.0
(1,6): -0.0
(1,7): 0.0
(1,8): -0.0
(1,18): -0.0
(1,19): 0.0
(2,3): -0.0
(2,4): 0.0
(2,5): -0.0
(2,6): 0.0
(2,7): -0.0
(2,8): 0.0
(2,9): -0.0
(2,10): 0.0
(2,11): -0.0
(2,18): 0.0
(2,19): -0.0
(3,4): -0.0
(3,5): 0.0
(3,6): -0.0
(3,8): -0.0
(3,9): 0.0
(3,10): -0.0
(3,11): 0.0
(3,18): -0.0
(4,5): -156.45926323781043
(4,8): 0.06691983108572115
(4,9): -0.0
(4,10): 0.0
(4,11): -0.0
(5,8): -0.06734579109841007
(5,9): 0.0
(5,10): -0.0
(5,11): 0.0
(6,7): -0.0
(6,8): 1.8598834102795192
(6,9): -0.0
(6,12): 1.6251394207813572
(6,13): -1.7376904462957734
(6,14): 0.06691983108572115
(6,15): -0.06734579109841007
(6,18): 0.15247178885977722
(6,19): -0.0
(7,8): -0.0
(7,9): 0.0
(7,12): -0.0
(7,13): 0.0
(7,14): -0.0
(7,15): 0.0
(7,18): -0.0
(8,9): -0.0
(8,10): 0.0
(8,11): -0.0
(8,12): 0.06691983108572115
(8,13): -0.06734579109841007
(8,14): 1.6251394207813572
(8,15): -1.73769044

-467.5821725068896

In [10]:
energy_lj + energy_coulomb

-467.5821725068896

In [11]:
forces[4]

array([-106.86795461, -107.03376857, -106.95256516])

In [12]:
platform = openmm.Platform.getPlatformByName('CUDA')
force = openmm.NonbondedForce()
force.setCutoffDistance(cutoff)
force.setUseDispersionCorrection(False)
force.setReactionFieldDielectric(1.0)
force.setNonbondedMethod(force.CutoffNonPeriodic)
system = openmm.System()
for i in range(positions.shape[0]):
    force.addParticle(charge(i)*(lambda_coulomb[1] if subsets[i] == 1 else 1), 1.0, eps)
    system.addParticle(1.0)
system.addForce(force)
integrator = openmm.VerletIntegrator(0.001)
context = openmm.Context(system, integrator, platform)
context.setPositions(positions)

OpenMMException: No compatible CUDA device is available

In [13]:
state = context.getState(getForces=True, getEnergy=True)
state.getPotentialEnergy()

NameError: name 'context' is not defined

In [14]:
ONE_4PI_EPS0 = 138.935457644382
REACTION_FIELD_K = 0.0
REACTION_FIELD_C = 1/cutoff
energy_coulomb = 0
forces = np.zeros((distances.shape[0], 3))
for i in range(distances.shape[0]):
    si = subsets[i]
    qi = charge(i)*(lambda_coulomb if subsets[i] == 1 else 1)
    for j in range(i+1, distances.shape[0]):
        sj = subsets[j]
        qj = charge(j)*(lambda_coulomb if subsets[j] == 1 else 1)
        r2 = distances[i, j]**2
        if distances[i, j] < cutoff:
            prefactor = ONE_4PI_EPS0*qi*qj
            energy_coulomb += prefactor*(1.0/distances[i, j] + REACTION_FIELD_K*r2 - REACTION_FIELD_C)
            fij = prefactor*(1.0/distances[i, j]**3 + REACTION_FIELD_K)
            nij = positions[i] - positions[j]
            if i == 4 or j == 4:
                print(f"fij[{i},{j}]: {fij} (dist: {distances[i, j]})")
            forces[i] += fij*nij
            forces[j] -= fij*nij
energy_coulomb

TypeError: can't multiply sequence by non-int of type 'float'

In [15]:
energies={
(1,0): (-156.459263)*(0.000000),
(2,0): (1.859875)*(0.000000),
(3,0): (-1.737683)*(0.000000),
(8,0): (0.152466)*(0.000000),
(9,0): (-0.067340)*(0.000000),
(10,0): (1.859875)*(0.000000),
(11,0): (-1.737683)*(0.000000),
(18,0): (1.859875)*(0.000000),
(19,0): (-1.737683)*(0.000000),
(0,1): (-156.459263)*(0.000000),
(2,1): (-1.737683)*(0.000000),
(3,1): (1.625132)*(0.000000),
(8,1): (-0.067340)*(0.000000),
(10,1): (-1.737683)*(0.000000),
(11,1): (1.625132)*(0.000000),
(18,1): (-1.737683)*(0.000000),
(19,1): (1.625132)*(0.000000),
(0,2): (1.859875)*(0.000000),
(1,2): (-1.737683)*(0.000000),
(3,2): (-156.459263)*(0.000000),
(4,2): (1.625132)*(0.000000),
(5,2): (-1.737683)*(0.000000),
(6,2): (0.066914)*(0.000000),
(7,2): (-0.067340)*(0.000000),
(8,2): (1.859875)*(0.000000),
(9,2): (-1.737683)*(0.000000),
(10,2): (0.152466)*(0.000000),
(11,2): (-0.067340)*(0.000000),
(18,2): (0.152466)*(0.000000),
(19,2): (-0.067340)*(0.000000),
(0,3): (-1.737683)*(0.000000),
(1,3): (1.625132)*(0.000000),
(2,3): (-156.459263)*(0.000000),
(4,3): (-1.737683)*(0.000000),
(5,3): (1.859875)*(0.000000),
(6,3): (-0.067340)*(0.000000),
(7,3): (0.066914)*(0.000000),
(8,3): (-1.737683)*(0.000000),
(9,3): (1.625132)*(0.000000),
(10,3): (-0.067340)*(0.000000),
(18,3): (-0.067340)*(0.000000),
(2,4): (1.625132)*(0.000000),
(3,4): (-1.737683)*(0.000000),
(5,4): (-156.459263)*(1.000000),
(6,4): (1.859875)*(1.000000),
(7,4): (-1.737683)*(0.000000),
(8,4): (0.066914)*(1.000000),
(9,4): (-0.067340)*(0.000000),
(2,5): (-1.737683)*(0.000000),
(3,5): (1.859875)*(0.000000),
(4,5): (-156.459263)*(1.000000),
(6,5): (-1.737683)*(1.000000),
(7,5): (1.625132)*(0.000000),
(8,5): (-0.067340)*(1.000000),
(9,5): (0.066914)*(0.000000),
(2,6): (0.066914)*(0.000000),
(3,6): (-0.067340)*(0.000000),
(4,6): (1.859875)*(1.000000),
(5,6): (-1.737683)*(1.000000),
(7,6): (-156.459263)*(0.000000),
(8,6): (1.625132)*(1.000000),
(9,6): (-1.737683)*(0.000000),
(12,6): (1.625132)*(1.000000),
(15,6): (-0.067340)*(1.000000),
(17,6): (-1.737683)*(0.000000),
(2,7): (-0.067340)*(0.000000),
(3,7): (0.066914)*(0.000000),
(4,7): (-1.737683)*(0.000000),
(5,7): (1.625132)*(0.000000),
(6,7): (-156.459263)*(0.000000),
(8,7): (-1.737683)*(0.000000),
(9,7): (1.859875)*(0.000000),
(12,7): (-1.737683)*(0.000000),
(14,7): (-0.067340)*(0.000000),
(15,7): (0.152466)*(0.000000),
(17,7): (1.859875)*(0.000000),
(0,8): (0.152466)*(0.000000),
(1,8): (-0.067340)*(0.000000),
(2,8): (1.859875)*(0.000000),
(3,8): (-1.737683)*(0.000000),
(4,8): (0.066914)*(1.000000),
(5,8): (-0.067340)*(1.000000),
(6,8): (1.625132)*(1.000000),
(7,8): (-1.737683)*(0.000000),
(9,8): (-156.459263)*(0.000000),
(10,8): (1.859875)*(0.000000),
(11,8): (-1.737683)*(0.000000),
(13,8): (-0.067340)*(1.000000),
(14,8): (1.625132)*(1.000000),
(15,8): (-1.737683)*(1.000000),
(16,8): (0.066914)*(0.000000),
(17,8): (-0.067340)*(0.000000),
(0,9): (-0.067340)*(0.000000),
(2,9): (-1.737683)*(0.000000),
(3,9): (1.625132)*(0.000000),
(4,9): (-0.067340)*(0.000000),
(5,9): (0.066914)*(0.000000),
(6,9): (-1.737683)*(0.000000),
(7,9): (1.859875)*(0.000000),
(8,9): (-156.459263)*(0.000000),
(10,9): (-1.737683)*(0.000000),
(11,9): (1.625132)*(0.000000),
(12,9): (-0.067340)*(0.000000),
(13,9): (0.066914)*(0.000000),
(14,9): (-1.737683)*(0.000000),
(15,9): (1.859875)*(0.000000),
(16,9): (-0.067340)*(0.000000),
(17,9): (0.152466)*(0.000000),
(0,10): (1.859875)*(0.000000),
(1,10): (-1.737683)*(0.000000),
(2,10): (0.152466)*(0.000000),
(3,10): (-0.067340)*(0.000000),
(8,10): (1.859875)*(0.000000),
(9,10): (-1.737683)*(0.000000),
(11,10): (-156.459263)*(0.000000),
(13,10): (-1.737683)*(0.000000),
(14,10): (0.066914)*(0.000000),
(15,10): (-0.067340)*(0.000000),
(16,10): (1.625132)*(0.000000),
(18,10): (0.152466)*(0.000000),
(19,10): (-0.067340)*(0.000000),
(0,11): (-1.737683)*(0.000000),
(1,11): (1.625132)*(0.000000),
(2,11): (-0.067340)*(0.000000),
(8,11): (-1.737683)*(0.000000),
(9,11): (1.625132)*(0.000000),
(10,11): (-156.459263)*(0.000000),
(13,11): (1.859875)*(0.000000),
(14,11): (-0.067340)*(0.000000),
(15,11): (0.066914)*(0.000000),
(16,11): (-1.737683)*(0.000000),
(18,11): (-0.067340)*(0.000000),
(6,12): (1.625132)*(1.000000),
(7,12): (-1.737683)*(0.000000),
(9,12): (-0.067340)*(0.000000),
(14,12): (1.625132)*(1.000000),
(15,12): (-1.737683)*(1.000000),
(17,12): (-156.459263)*(0.000000),
(8,13): (-0.067340)*(1.000000),
(9,13): (0.066914)*(0.000000),
(10,13): (-1.737683)*(0.000000),
(11,13): (1.859875)*(0.000000),
(14,13): (-1.737683)*(1.000000),
(15,13): (1.625132)*(1.000000),
(16,13): (-156.459263)*(0.000000),
(7,14): (-0.067340)*(0.000000),
(8,14): (1.625132)*(1.000000),
(9,14): (-1.737683)*(0.000000),
(10,14): (0.066914)*(0.000000),
(11,14): (-0.067340)*(0.000000),
(12,14): (1.625132)*(1.000000),
(13,14): (-1.737683)*(1.000000),
(15,14): (-156.459263)*(1.000000),
(16,14): (1.859875)*(0.000000),
(17,14): (-1.737683)*(0.000000),
(6,15): (-0.067340)*(1.000000),
(7,15): (0.152466)*(0.000000),
(8,15): (-1.737683)*(1.000000),
(9,15): (1.859875)*(0.000000),
(10,15): (-0.067340)*(0.000000),
(11,15): (0.066914)*(0.000000),
(12,15): (-1.737683)*(1.000000),
(13,15): (1.625132)*(1.000000),
(14,15): (-156.459263)*(1.000000),
(16,15): (-1.737683)*(0.000000),
(17,15): (1.859875)*(0.000000),
(8,16): (0.066914)*(0.000000),
(9,16): (-0.067340)*(0.000000),
(10,16): (1.625132)*(0.000000),
(11,16): (-1.737683)*(0.000000),
(13,16): (-156.459263)*(0.000000),
(14,16): (1.859875)*(0.000000),
(15,16): (-1.737683)*(0.000000),
(6,17): (-1.737683)*(0.000000),
(7,17): (1.859875)*(0.000000),
(8,17): (-0.067340)*(0.000000),
(9,17): (0.152466)*(0.000000),
(12,17): (-156.459263)*(0.000000),
(14,17): (-1.737683)*(0.000000),
(15,17): (1.859875)*(0.000000),
(0,18): (1.859875)*(0.000000),
(1,18): (-1.737683)*(0.000000),
(2,18): (0.152466)*(0.000000),
(3,18): (-0.067340)*(0.000000),
(10,18): (0.152466)*(0.000000),
(11,18): (-0.067340)*(0.000000),
(19,18): (-156.459263)*(0.000000),
(0,19): (-1.737683)*(0.000000),
(1,19): (1.625132)*(0.000000),
(2,19): (-0.067340)*(0.000000),
(10,19): (-0.067340)*(0.000000),
(18,19): (-156.459263)*(0.000000),
}
energies

{(1, 0): -0.0,
 (2, 0): 0.0,
 (3, 0): -0.0,
 (8, 0): 0.0,
 (9, 0): -0.0,
 (10, 0): 0.0,
 (11, 0): -0.0,
 (18, 0): 0.0,
 (19, 0): -0.0,
 (0, 1): -0.0,
 (2, 1): -0.0,
 (3, 1): 0.0,
 (8, 1): -0.0,
 (10, 1): -0.0,
 (11, 1): 0.0,
 (18, 1): -0.0,
 (19, 1): 0.0,
 (0, 2): 0.0,
 (1, 2): -0.0,
 (3, 2): -0.0,
 (4, 2): 0.0,
 (5, 2): -0.0,
 (6, 2): 0.0,
 (7, 2): -0.0,
 (8, 2): 0.0,
 (9, 2): -0.0,
 (10, 2): 0.0,
 (11, 2): -0.0,
 (18, 2): 0.0,
 (19, 2): -0.0,
 (0, 3): -0.0,
 (1, 3): 0.0,
 (2, 3): -0.0,
 (4, 3): -0.0,
 (5, 3): 0.0,
 (6, 3): -0.0,
 (7, 3): 0.0,
 (8, 3): -0.0,
 (9, 3): 0.0,
 (10, 3): -0.0,
 (18, 3): -0.0,
 (2, 4): 0.0,
 (3, 4): -0.0,
 (5, 4): -156.459263,
 (6, 4): 1.859875,
 (7, 4): -0.0,
 (8, 4): 0.066914,
 (9, 4): -0.0,
 (2, 5): -0.0,
 (3, 5): 0.0,
 (4, 5): -156.459263,
 (6, 5): -1.737683,
 (7, 5): 0.0,
 (8, 5): -0.06734,
 (9, 5): 0.0,
 (2, 6): 0.0,
 (3, 6): -0.0,
 (4, 6): 1.859875,
 (5, 6): -1.737683,
 (7, 6): -0.0,
 (8, 6): 1.625132,
 (9, 6): -0.0,
 (12, 6): 1.625132,
 (15, 6): -0.0

In [32]:
import itertools
n = distances.shape[0]
assert all((i, j) in energies for (j, i) in energies)
for i, j in itertools.combinations(range(n), 2):
    if (i, j) in energies and energies[(i, j)] != -0.0:
        print(f"energy({i}, {j}): {energies[(i, j)]} {energies[(j, i)]}")
        if (i, j) in computed_energies:
            print(f"computed_energy({i}, {j}): {computed_energies[(i, j)]}")
        if (j, i) in computed_energies:
            print(f"computed_energy({j}, {i}): {computed_energies[(j, i)]}")
    else:
        if (i, j) in computed_energies and computed_energies[(i, j)] != -0.0:
            print(f"computed_energy({i}, {j}): {computed_energies[(i, j)]}")
        if (j, i) in computed_energies and computed_energies[(j, i)] != -0.0:
            print(f"computed_energy({j}, {i}): {computed_energies[(j, i)]}")


energy(4, 5): -156.459263 -156.459263
computed_energy(4, 5): -156.45926323781043
energy(4, 6): 1.859875 1.859875
energy(4, 8): 0.066914 0.066914
computed_energy(4, 8): 0.06691983108572115
energy(5, 6): -1.737683 -1.737683
energy(5, 8): -0.06734 -0.06734
computed_energy(5, 8): -0.06734579109841007
energy(6, 8): 1.625132 1.625132
computed_energy(6, 8): 1.8598834102795192
energy(6, 12): 1.625132 1.625132
computed_energy(6, 12): 1.6251394207813572
computed_energy(6, 13): -1.7376904462957734
computed_energy(6, 14): 0.06691983108572115
energy(6, 15): -0.06734 -0.06734
computed_energy(6, 15): -0.06734579109841007
computed_energy(6, 18): 0.15247178885977722
computed_energy(8, 12): 0.06691983108572115
energy(8, 13): -0.06734 -0.06734
computed_energy(8, 13): -0.06734579109841007
energy(8, 14): 1.625132 1.625132
computed_energy(8, 14): 1.6251394207813572
energy(8, 15): -1.737683 -1.737683
computed_energy(8, 15): -1.7376904462957734
computed_energy(12, 13): -156.45926323781043
energy(12, 14): 1.62

True

In [ ]:
computed_energies

-467.5821725068896

In [270]:
import pandas as pd

# Create lists to store the data
i_values = []
j_values = []
computed_energy_values = []
reference_energy_values = []

# Iterate through all pairs in computed_energies
for (i, j) in computed_energies.keys():
    i_values.append(i)
    j_values.append(j)
    computed_energy_values.append(computed_energies[(i, j)] if (i, j) in computed_energies else computed_energies[(j, i)])
    reference_energy_values.append(energies[(i, j)])

# Create the dataframe
df = pd.DataFrame({
    'i': i_values,
    'j': j_values,
    'computed_energy': computed_energy_values,
    'reference_energy': reference_energy_values
})

df

KeyError: (0, 6)